In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Dependencies

In [2]:
!pip install datasets torch accelerate>=0.21.0 transformers==4.28 evaluate seqeval sacrebleu requests==2.31.0
# rouge_score
# sacrebleu

## Import libraries

In [3]:
import evaluate
import logging
import numpy as np
import os
import torch
from datasets import (
    load_dataset,
    load_metric,
)
from transformers import (
    DataCollatorForLanguageModeling,
    AutoTokenizer,
    AutoModelForCausalLM,
    TextDataset,
    TrainingArguments,
    Trainer,
    pipeline,
)
from google.colab import userdata
from huggingface_hub import login

In [5]:
login(userdata.get('HUGGING_FACE'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Prepare data

### Load Dataset

In [6]:
dataset = load_dataset("json", data_files="/content/drive/MyDrive/models/dataset_sms_generation.jsonl")
dataset = dataset.shuffle()
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 2252
    })
})

In [7]:
dataset["train"][2]

{'prompt': 'Domina el inglés con nosotros. Descuentos especiales por inscripción anticipada referir a un amigo y planes familiares. Aprovecha esta oportunidad única.'}

In [8]:
dataset = dataset['train'].train_test_split(test_size=0.20)
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 1801
    })
    test: Dataset({
        features: ['prompt'],
        num_rows: 451
    })
})

### Load model and tokenizer

Modelo: [datificate/gpt2-small-spanish](https://huggingface.co/datificate/gpt2-small-spanish)

In [9]:
# model_card = 'gpt2'
# model_card = 'distilbert/distilgpt2'
# model_card = 'EleutherAI/gpt-neo-125M'

model_card = 'datificate/gpt2-small-spanish'

model = AutoModelForCausalLM.from_pretrained(model_card, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_card)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

### Tokenize the data

In [10]:
def tokenize_fn(examples):
  # Add a special token as eos_token at the end of each sequence.
  # This helps the model to understand when a sequence ends and another begins.
  texts = [text + tokenizer.eos_token for text in examples['prompt']]
  tokenized_inputs = tokenizer(
      texts,
      padding='max_length',
      truncation=True,
      max_length=160,
      return_tensors="pt",
  )

  # Setup the tokenizer for targets. They are the same as the input text.
  labels = tokenized_inputs['input_ids']
  tokenized_inputs['labels'] = labels

  return tokenized_inputs

In [11]:
# Tokenize all dataset
tokenized_dataset = dataset.map(tokenize_fn, batched=True)
tokenized_dataset

Map:   0%|          | 0/1801 [00:00<?, ? examples/s]

Map:   0%|          | 0/451 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1801
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 451
    })
})

### Data Collator

In [12]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)  # batch!!!

## Training

### Setup Logging

In [14]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### Set Up Training Hyperparameters

In [15]:
batch_size = 64
# Show the training loss with every epoch
logging_steps = len(tokenized_dataset["train"]) // batch_size

training_args = TrainingArguments(
    output_dir='./results_sms_generation',
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=logging_steps,
    logging_dir='./logs',
    fp16=True,  # Use mixed precision training helps to save memory and time
)

### Set up Trainer and execute

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.629100,1.547054
2,1.320400,1.340690
3,1.112500,1.257787
4,1.103300,1.232042


TrainOutput(global_step=3604, training_loss=1.486849077269187, metrics={'train_runtime': 425.1739, 'train_samples_per_second': 16.944, 'train_steps_per_second': 8.477, 'total_flos': 588233687040000.0, 'train_loss': 1.486849077269187, 'epoch': 4.0})

In [17]:
eval_results = trainer.evaluate()
eval_results

{'eval_loss': 1.232041597366333,
 'eval_runtime': 4.3742,
 'eval_samples_per_second': 103.105,
 'eval_steps_per_second': 51.667,
 'epoch': 4.0}

In [18]:
import math
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 3.43


## Save the fine tuned model

In [19]:
my_fine_tuned_model = "/content/drive/MyDrive/models/fine-tuned-sms-generation-model"
model.save_pretrained(my_fine_tuned_model)
tokenizer.save_pretrained(my_fine_tuned_model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Make predictions

## Con pipeline

In [23]:
generator_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)

texts = [
  "Descubre la belleza de Bali con nuestra agencia",
  "Disfruta de la mejor cocina en nuestros restaurantes",
  "Renueva tu armario con estilo y ahorro",
  "Habla como un nativo"
]

for text in texts:
  print(f"\nSMS a generar: {text}...")
  generated_text = generator_pipeline(
      text,
      num_return_sequences=3,
      early_stopping=True,
      no_repeat_ngram_size=2,  # evita repeticiones
  )
  for index, sms in enumerate(generated_text):
    print(f"-------- SMS{index + 1} --------")
    print(f"{sms['generated_text']} \n")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



SMS a generar: Descubre la belleza de Bali con nuestra agencia...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------- SMS1 --------
Descubre la belleza de Bali con nuestra agencia. Descuentos imperdibles en paquetes todo incluido. Reserva ya y vive una experiencia inolvidable. Viaja con nosotros. Inscríbete ya. No te lo pierdas. Te esperamos. Aprovecha. Mañana logra. 

-------- SMS2 --------
Descubre la belleza de Bali con nuestra agencia y obtén hasta un 30% de descuento en paquetes turísticos. Aprovecha nuestras ofertas y vive una experiencia inolvidable. Viaja al mejor precio. Reserva ya. No dejes pasar este encanto. Planifica tu próximo viaje con nosotros. destinos 

-------- SMS3 --------
Descubre la belleza de Bali con nuestra agencia. Descuentos exclusivos en paquetes todo incluido. Reserva ahora y vive una experiencia inolvidable. Viaja sin preocupaciones. Solo por tiempo limitado. No te lo pierdas. Aventura te espera. Te esperamos. Inscríbete ya. 


SMS a generar: Disfruta de la mejor cocina en nuestros restaurantes...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------- SMS1 --------
Disfruta de la mejor cocina en nuestros restaurantes. Hoy martes 15% de descuento en platillos seleccionados. Por último miércoles 10% en postres y vinos al mejor precio. Te esperamos. No te lo pierdas. Deléitate. Reserva ya. Hazte con nosotros. Inscrí 

-------- SMS2 --------
Disfruta de la mejor cocina en nuestros restaurantes. 15% de descuento en platos principales 20% en postres y 15x1 en bebidas. Ven y saborea la experiencia. Te esperamos. No te lo pierdas. Inscríbete ya. Reserva ahora. Visítanos hoy 

-------- SMS3 --------
Disfruta de la mejor cocina en nuestros restaurantes con descuentos especiales en todos los platos de temporada. Reserva ahora y luce espectacular con un 20% de descuento en cada comida. Te esperamos. Hazte. Inscríbete ya. No te lo pierdas. Ven a disfrutar. 


SMS a generar: Renueva tu armario con estilo y ahorro...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------- SMS1 --------
Renueva tu armario con estilo y ahorro en nuestra tienda de ropa. Descuentos de hasta 50% en jeans 2x1 en camisetas y rebajas de un 50%. No te lo pierdas. Visita ya. No pierdes. Viaja sin límites. Reserva ya tu 

-------- SMS2 --------
Renueva tu armario con estilo y ahorro. Descuentos de hasta 50% en toda la colección. Visítanos hoy y descubre esta colección de ropa. No te lo pierdas. Solo por hoy. Ven y sorpréndete. Te esperamos. Mañana te sorprende. 

-------- SMS3 --------
Renueva tu armario con estilo y ahorro. Descuentos de hasta 50% en toda la colección de primavera. Visítanos hoy y luce increíble. No te lo pierdas. Ven ya. 2x1 en jeans. La moda de moda femenina lo reserva. Compra ahora. 


SMS a generar: Habla como un nativo...
-------- SMS1 --------
Habla como un nativo. Descuentos especiales en matrícula y mensualidades. Aprende de forma divertida y efectiva con nosotros. Inscríbete ya. No te lo pierdas. Te esperamos. Disfruta. Ahorra 15% en nuestra aca